In [1]:
import sys
import json
import os
import traceback

def extract_printable_strings(path, minlen=4):
    """Fallback: pull readable ASCII sequences from a binary file."""
    out = []
    with open(path, "rb") as f:
        buf = bytearray()
        while True:
            b = f.read(1)
            if not b:
                if len(buf) >= minlen:
                    out.append(buf.decode("ascii", errors="ignore"))
                break
            v = b[0]
            if 32 <= v < 127:
                buf.append(v)
            else:
                if len(buf) >= minlen:
                    out.append(buf.decode("ascii", errors="ignore"))
                buf = bytearray()
    return out

In [2]:
# Parameters
path = "/data/gardeux/MoonVale_SaveEditor/PersData.kat"

if not os.path.exists(path):
    print(f"File not found: {path}", file=sys.stderr)
    sys.exit(1)

In [3]:
import pythonnet  # type: ignore
pythonnet.load()  # loads CoreCLR if present; no-op on pythonnet 2

In [4]:
import clr  # provided by pythonnet

In [5]:
clr.AddReference("System")
clr.AddReference("System.Core")
clr.AddReference("Microsoft.CSharp")

In [6]:
from Microsoft.CSharp import CSharpCodeProvider
from System.CodeDom.Compiler import CompilerParameters

In [7]:
cs = r'''
using System;
using System.Collections;
using System.Collections.Generic;
using System.Runtime.Serialization;

namespace KatReader {
  [Serializable]
  public class Node : ISerializable {
    public string DotNetType;
    public Dictionary<string, object> Data = new Dictionary<string, object>();

    public Node() {}

    protected Node(SerializationInfo info, StreamingContext context) {
      this.DotNetType = info.FullTypeName;
      foreach (SerializationEntry e in info) {
        Data[e.Name] = e.Value;
      }
    }
    public void GetObjectData(SerializationInfo info, StreamingContext context) {
      throw new NotSupportedException("Dump-only");
    }
  }

  public class RedirectBinder : SerializationBinder {
    public override Type BindToType(string assemblyName, string typeName) {
      try {
        if (!string.IsNullOrEmpty(typeName) && typeName.StartsWith("Everbyte.TextGame.Saving"))
          return typeof(Node);
        if (!string.IsNullOrEmpty(assemblyName) && assemblyName.StartsWith("Everbyte.TextGame.Saving"))
          return typeof(Node);

        var full = string.IsNullOrEmpty(assemblyName) ? typeName : (typeName + ", " + assemblyName);
        var t = Type.GetType(full, throwOnError:false);
        if (t != null) return t;

        if (!string.IsNullOrEmpty(typeName) && typeName.IndexOf("Everbyte", StringComparison.OrdinalIgnoreCase) >= 0)
          return typeof(Node);

        return typeof(object);
      } catch {
        return typeof(object);
      }
    }
  }
}
'''
provider = CSharpCodeProvider()
parms = CompilerParameters()
parms.GenerateInMemory = True
parms.GenerateExecutable = False
parms.TreatWarningsAsErrors = False
parms.ReferencedAssemblies.Add("System.dll")
parms.ReferencedAssemblies.Add("System.Core.dll")
parms.ReferencedAssemblies.Add("mscorlib.dll")  # Mono holds BinaryFormatter here
res = provider.CompileAssemblyFromSource(parms, cs)
if res.Errors.HasErrors:
    print("\n".join(str(e) for e in res.Errors), file=sys.stderr)
    sys.exit(2)

asm = res.CompiledAssembly
NodeType = asm.GetType("KatReader.Node")
RedirectBinderType = asm.GetType("KatReader.RedirectBinder")

In [11]:
# Now set up BinaryFormatter with our binder
from System.IO import FileStream, FileMode, FileAccess
from System.Runtime.Serialization.Formatters.Binary import BinaryFormatter
from System import Activator

In [12]:
bf = BinaryFormatter()
# Instantiate RedirectBinder correctly:
binder = Activator.CreateInstance(RedirectBinderType)
bf.Binder = binder

In [13]:
# Deserialize
fs = FileStream(path, FileMode.Open, FileAccess.Read)
root = bf.Deserialize(fs)
fs.Close()

In [15]:
# --- Convert to Python ---
from System import DateTime, Decimal, Guid, Array, Byte, String
from System.Collections import IDictionary, IEnumerable
from System.Reflection import BindingFlags
from System.Runtime.CompilerServices import RuntimeHelpers

seen = set()

def to_py(o):
    if o is None: return None
    if isinstance(o, (str, int, float, bool)): return o

    try:
        oid = RuntimeHelpers.GetHashCode(o)
        if oid in seen: return None
        seen.add(oid)
    except Exception:
        pass

    try:
        tname = str(o.GetType().FullName)
    except Exception:
        tname = None

    if tname == "KatReader.Node":
        py = {"$original_dotnet_type": str(getattr(o, "DotNetType", ""))}
        try:
            data = getattr(o, "Data")
            dpy = {}
            for k in data.Keys:
                dpy[str(k)] = to_py(data[k])
            py.update(dpy)
        except Exception:
            pass
        return py

    try:
        t = o.GetType()
        if t.IsPrimitive: return o
    except Exception:
        pass

    if isinstance(o, DateTime): return o.ToString("o")
    if isinstance(o, Decimal):  return float(o)
    if isinstance(o, Guid):     return str(o)

    try:
        if isinstance(o, Array) and o.GetType().GetElementType() in (Byte,):
            return [int(b) for b in o]
    except Exception:
        pass

    if isinstance(o, IDictionary):
        d = {}
        try:
            for k in o.Keys:
                d[str(k)] = to_py(o[k])
        except Exception:
            for entry in o:
                try:
                    d[str(entry.Key)] = to_py(entry.Value)
                except Exception:
                    pass
        return d

    if isinstance(o, IEnumerable) and not isinstance(o, String):
        lst = []
        try:
            it = o.GetEnumerator()
            while it.MoveNext():
                lst.append(to_py(it.Current))
        except Exception:
            try:
                for it in o:
                    lst.append(to_py(it))
            except Exception:
                pass
        return lst

    pyobj = {}
    try:
        pyobj["$type"] = tname
        flags = BindingFlags.Instance | BindingFlags.Public | BindingFlags.NonPublic
        for f in o.GetType().GetFields(flags):
            try:
                pyobj[f.Name] = to_py(f.GetValue(o))
            except Exception:
                pass
    except Exception:
        try:
            return str(o)
        except Exception:
            return None
    return pyobj

json.dump(to_py(root), sys.stdout, ensure_ascii=False, indent=2)

{
  "$original_dotnet_type": "KatReader.Node",
  "currentChapter": 1,
  "currentGameState": 1,
  "gameInitCounter": 8,
  "chapterInitCounter": 2,
  "storyProgress": 0,
  "boughtCounter": 5,
  "spendCounter": 62,
  "replayCounters": [
    0
  ],
  "saveGameVersion": 1,
  "lastGameVersionSaved": "1.2.5",
  "coins": 35,
  "diamonds": 99343,
  "energy": 5,
  "lastEnergyUpdate": "",
  "boughtProductIds": [],
  "conditions": [],
  "characters": [
    "char_02a1000",
    "char_01a1000"
  ],
  "paths": {
    "$original_dotnet_type": "KatReader.Node",
    "_items": [
      {
        "$original_dotnet_type": "KatReader.Node",
        "uID": "path_002",
        "cptrMrk": "a",
        "activeState": 0,
        "stateID": "",
        "memberIDs": [
          "char_02"
        ],
        "lastSeenNodeID": "4536",
        "pathHistory": [
          "aT452714:37",
          "aP453614:37",
          "aT450614:37"
        ]
      },
      null,
      null,
      null
    ],
    "_size": 1,
    "_versio